# Convolutionクラスのforwardメソッドの理解(im2colを含む)
掲題の件について、テキストの実装を参考に自分自身でサンプルコードを作り追体験をすることで、畳込み演算について理解を深める。自分の実力的にここは難関なので、forward内で行われているim2colとreshapeを中心に調査してみる。
1. まずはim2colを動作させてみる
2. col展開でなぜ畳み込み演算が実現できるか、重みとフィルターの関係性について調べる

## 1. まずはim2colを動作させてみよう

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from common.util import *

まずは、サンプル行列を生成する汎用的な関数を用意しておこう。以下のinit_sample_matrixは指定したサイズの行列を生成する。各要素の数字の位には意味がある。

1. 千の位：画像番号、または、フィルタ番号
2. 百の位：チャネル番号
3. 十の位：行番号
4. 一の位：列番号

なお、1スタートする(プログラミング上の配列は0スタートだが、この関数では1スタートを採用)

In [2]:
#サンプル行列の初期化
#init_sample_matrixは指定したサイズの行列を生成する。各要素の十の位が行番号を示し、一の位が列番号を示す。
def init_sample_matrix(filter_num=0, channel=0, height=6, width = 8):
    matrix = []

    for row in range(height):
        temp_row = []
        for col in range(width):
            elem = (row+1)*10 + col+1
            elem += channel * 100
            elem += filter_num * 1000
            temp_row.append(elem)
        matrix.append(temp_row)

    return np.array(matrix)

In [3]:
m = init_sample_matrix()
print(m)

[[11 12 13 14 15 16 17 18]
 [21 22 23 24 25 26 27 28]
 [31 32 33 34 35 36 37 38]
 [41 42 43 44 45 46 47 48]
 [51 52 53 54 55 56 57 58]
 [61 62 63 64 65 66 67 68]]


次に以下のパラメータにて、im2colを動作させてみよう。

In [4]:
filter_size_x = 3
filter_size_y = 3
stride        = 1
pad           = 0

In [5]:
img = np.array([[m]])
print(img.shape)
col = im2col(img, filter_size_x, filter_size_y, stride, pad)
print(col)

(1, 1, 6, 8)
[[11. 12. 13. 21. 22. 23. 31. 32. 33.]
 [12. 13. 14. 22. 23. 24. 32. 33. 34.]
 [13. 14. 15. 23. 24. 25. 33. 34. 35.]
 [14. 15. 16. 24. 25. 26. 34. 35. 36.]
 [15. 16. 17. 25. 26. 27. 35. 36. 37.]
 [16. 17. 18. 26. 27. 28. 36. 37. 38.]
 [21. 22. 23. 31. 32. 33. 41. 42. 43.]
 [22. 23. 24. 32. 33. 34. 42. 43. 44.]
 [23. 24. 25. 33. 34. 35. 43. 44. 45.]
 [24. 25. 26. 34. 35. 36. 44. 45. 46.]
 [25. 26. 27. 35. 36. 37. 45. 46. 47.]
 [26. 27. 28. 36. 37. 38. 46. 47. 48.]
 [31. 32. 33. 41. 42. 43. 51. 52. 53.]
 [32. 33. 34. 42. 43. 44. 52. 53. 54.]
 [33. 34. 35. 43. 44. 45. 53. 54. 55.]
 [34. 35. 36. 44. 45. 46. 54. 55. 56.]
 [35. 36. 37. 45. 46. 47. 55. 56. 57.]
 [36. 37. 38. 46. 47. 48. 56. 57. 58.]
 [41. 42. 43. 51. 52. 53. 61. 62. 63.]
 [42. 43. 44. 52. 53. 54. 62. 63. 64.]
 [43. 44. 45. 53. 54. 55. 63. 64. 65.]
 [44. 45. 46. 54. 55. 56. 64. 65. 66.]
 [45. 46. 47. 55. 56. 57. 65. 66. 67.]
 [46. 47. 48. 56. 57. 58. 66. 67. 68.]]


参考URL1の記事で説明されているように、畳み込みのフィルターの範囲の要素を並べて各行(行ベクトル)としていき、それぞstride幅ずつストライドしていき、col展開する（参考URLの用語を流用)（なお、col2imも同様どのこと。ただし、こちらはimage表現に直した際に各ピクセルの値を加算していく所が特色的。)
例を用いて少し詳しく補足する。まず、上記のinit_sample_matrixで生成した行列mは以下の内容だった。

In [6]:
#[[11 12 13 14 15 16 17 18]
# [21 22 23 24 25 26 27 28]
# [31 32 33 34 35 36 37 38]
# [41 42 43 44 45 46 47 48]
# [51 52 53 54 55 56 57 58]
# [61 62 63 64 65 66 67 68]]

フィルタサイズは3×3のため最初の畳み込み演算の範囲（一部）は以下になる。

In [7]:
#[[11 12 13 ]
# [21 22 23 ]
# [31 32 33 ]]

im2colの出力結果は上記範囲を１つの行ベクトルにまとめたものが並んでいることがわかる。

In [8]:
# [11. 12. 13. 21. 22. 23. 31. 32. 33.]

あとは同様といった動作である。

In [9]:
#※読み飛ばしても構わない
#参考：col2imを試しに少し動作してみる。詳しい解析、解説記事は別途作成する。
print(img.shape)

img_ret = col2im(col, img.shape, filter_size_x, filter_size_y, stride, pad)
print(img_ret.shape)
print(img_ret)

(1, 1, 6, 8)
(1, 1, 6, 8)
[[[[ 11.  24.  39.  42.  45.  48.  34.  18.]
   [ 42.  88. 138. 144. 150. 156. 108.  56.]
   [ 93. 192. 297. 306. 315. 324. 222. 114.]
   [123. 252. 387. 396. 405. 414. 282. 144.]
   [102. 208. 318. 324. 330. 336. 228. 116.]
   [ 61. 124. 189. 192. 195. 198. 134.  68.]]]]


## 2. col展開でなぜ畳み込み演算が実現できるか、重みとフィルターの関係性について調べる
入力行列をim2colを実施してcol展開すると、畳込み演算が行列の積で済むということだが、正直ピンと来ない。そこで、もう少し詳しく解析をすすめるため、テキストのConvolutionクラスを参考にしてみる。この辺は数式がテキストで紹介されていないため、コードを直接解析して理解をすすめる必要がある。
### 重み込演算コードの観察(common/layers.pyより)

In [10]:
#common/layers.pyより関係する所のみを抜粋
class Convolution:
    #(中略)
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out


入力行列xをim2colでcol展開した後、加工したW(col_W)と行列colで行列積(col * colW)をとっている。
上記col展開の具体的な観察を踏まえ、なぜ、colとcol_W(すなわちcol_W = self.W.reshape(FN, -1).T)の行列積を取るコトが、畳み込み演算に繋がるのかを考察する。im2colで実施している内容はすでにわかったため、self.W.reshape(FN, -1).Tについて解析をすすめる。

### "self.W.reshape(FN, -1).T"って何やっているのか？ステップバイステップで動作を追う

字面上は重みWをreshapeで変形したものの転置をとっている。テキストP217を見ると、入力データ、フィルター（重み）、出力データはそれぞれ以下の次元だった。

1. 入力データ：(C,H,W)。Cはチャンネル数、Hは高さ、Wは幅
2. フィルター：(FN,C,FH,FW)。FNはフィルターの数、Cはチャンネル数(入力のチャンネル数と同一値）、FH、FWはそれぞれフィルターの高さと幅
3. 出力データ：(FN,OH,OW)。FNはフィルターの数、OH、OWはそれぞれ、出力データの高さと幅。

ここで、W(フィルータ)は(FN,C,FH,FW)の4次元配列になっている。また、この時、Wに対してreshape(FN, -1)を施すとどうなるのか？
実際にサンプルコードを作りながら試していく。まずは、xとWを適当に用意してみよう。なお、思考を簡略化するために、まずは、paddingが0、strideが1の場合で単純に考えてみる。
なお、テキストで用意されているSimpleConvNetおよび、im2colは入力データが4次元(N,C,H,W)になることを前提としているため、今から作るコードもそれに合わせたデータ構造とする。なお、load_mnistではこの辺のクラスの仕様との整合性をflatten引数によって、挙動を調整している。参考までに動作を乗せておく。

### 参考動作(load_mnistのflatten引数)：※読み飛ばしても構わない

In [11]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み(flatten=Falseであって、SimpleConvNet用途の場合)
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

print(x_train.shape)
print(x_train[0].shape)

# データの読み込み(flatten=Trueであって、その他用途の場合)
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True)

print(x_train.shape)
print(x_train[0].shape)

(60000, 1, 28, 28)
(1, 28, 28)
(60000, 784)
(784,)


### サンプルの全体的なパラメータ
今後のために定めておく
1. パディング、ストライド(pad,stride)
2. 入力画像の高さ、幅、および、チャネル数。画像の枚数　※理解/説明の便宜上、ここでデータは画像を想定するが、もちろん、データは画像かどうかは問わない。
3. フィルタの高さ、幅
4. フィルタの枚数(=画像の枚数)
5. 出力データ（画像）の高さと幅（1.~4.から自動的に計算される。）

In [12]:
#1. パディング、ストライド(pad,stride)
pad    = 0
stride = 1
#2. 入力画像の高さ、幅、および、チャネル数。画像の枚数
H  = 4
W  = 5
C  = 1 #上記参考にも合ったが、mnistはチャンネル数1のようなので、このサンプルテストもそれに合わせる
N  = 2 #入力画像の枚数
#3. フィルタの高さ、幅
FH = 3
FW = 3
#4. フィルタの枚数(=画像の枚数)
FN = 2 #フィルタの枚数
# 5. 出力データ（画像）の高さと幅（1.~4.から自動的に計算される。）
out_h = 1 + int((H + 2*pad - FH) / stride)
out_w = 1 + int((W + 2*pad - FW) / stride)

print("出力画像の高さ=%d と 幅=%d" % (out_h ,out_w))

出力画像の高さ=2 と 幅=3


### 入力データ(x)と重み(W)を用意する

In [13]:
#xの用意(mnistはチャンネル数1のようなので、このテストもそれに合わせる)
print("=== preparing of x===")
x1 = init_sample_matrix(filter_num = 1, channel=C, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x2 = init_sample_matrix(filter_num = 2, channel=C, height=H, width=W)
x = np.array([[x1],[x2]])
print(x.shape)
print(x)
#Wの用意(mnistはチャンネル数1のようなので、このテストもそれに合わせる)
print("=== preparing of W===")

w1 = init_sample_matrix(filter_num = 3, channel=C, height=FH, width=FW) #フィルタの枚数が3枚という訳ではないが、入力と重みの数値を区別するためにあえて3に設定
w2 = init_sample_matrix(filter_num = 4, channel=C, height=FH, width=FW) #同様の理由で4に設定
W = np.array([[w1],[w2]])
assert FN == W.shape[0], "FN ERROR"
print(W.shape)
print(W)

=== preparing of x===
(2, 1, 4, 5)
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]
=== preparing of W===
(2, 1, 3, 3)
[[[[3111 3112 3113]
   [3121 3122 3123]
   [3131 3132 3133]]]


 [[[4111 4112 4113]
   [4121 4122 4123]
   [4131 4132 4133]]]]


### 入力xをcol展開する(変数colに結果を格納)

In [14]:
#入力xをcol展開する
print("入力xのcol展開　★1")
print(x.shape)
col = im2col(x, FW, FH, stride, pad)
print(col.shape)
print(col)

入力xのcol展開　★1
(2, 1, 4, 5)
(12, 9)
[[1111. 1112. 1113. 1121. 1122. 1123. 1131. 1132. 1133.]
 [1112. 1113. 1114. 1122. 1123. 1124. 1132. 1133. 1134.]
 [1113. 1114. 1115. 1123. 1124. 1125. 1133. 1134. 1135.]
 [1121. 1122. 1123. 1131. 1132. 1133. 1141. 1142. 1143.]
 [1122. 1123. 1124. 1132. 1133. 1134. 1142. 1143. 1144.]
 [1123. 1124. 1125. 1133. 1134. 1135. 1143. 1144. 1145.]
 [2111. 2112. 2113. 2121. 2122. 2123. 2131. 2132. 2133.]
 [2112. 2113. 2114. 2122. 2123. 2124. 2132. 2133. 2134.]
 [2113. 2114. 2115. 2123. 2124. 2125. 2133. 2134. 2135.]
 [2121. 2122. 2123. 2131. 2132. 2133. 2141. 2142. 2143.]
 [2122. 2123. 2124. 2132. 2133. 2134. 2142. 2143. 2144.]
 [2123. 2124. 2125. 2133. 2134. 2135. 2143. 2144. 2145.]]


入力をcol展開した後は12行9列の行列になっている。画像毎に、畳込み演算対象のデータが列ベクトルとして並んでいることがよくわかる。画像の境界なしに、延べたんに展開しているのがポイントか。im2colはある意味単純。それでは、Wのreshapeについてはどうか。

### reshape(FN, -1)をかましてみる(変数col_Wに結果を格納)

In [15]:
print("FNの値%d" % (FN))

print("元のWの値")
print(W)
print("reshape(FN,-1)")
temp = W.reshape(FN, -1) #最初の次元だけをFNにして、あとは自動設定(-1)
print(temp.shape)  #結果として、2行18列の配列になる。
print(temp)

print("さらにそれを転置 ★2")
col_W = temp.T
print(col_W.shape)
print(col_W)
print(col_W[0,0])

FNの値2
元のWの値
[[[[3111 3112 3113]
   [3121 3122 3123]
   [3131 3132 3133]]]


 [[[4111 4112 4113]
   [4121 4122 4123]
   [4131 4132 4133]]]]
reshape(FN,-1)
(2, 9)
[[3111 3112 3113 3121 3122 3123 3131 3132 3133]
 [4111 4112 4113 4121 4122 4123 4131 4132 4133]]
さらにそれを転置 ★2
(9, 2)
[[3111 4111]
 [3112 4112]
 [3113 4113]
 [3121 4121]
 [3122 4122]
 [3123 4123]
 [3131 4131]
 [3132 4132]
 [3133 4133]]
3111


### ここまでの理解
まず、入力xと重みWの畳み込み演算について触れておく。まず、xとWのデータである。

In [16]:
#=== preparing of x===
#(2, 1, 4, 5)
#[[[[1111 1112 1113 1114 1115]
#   [1121 1122 1123 1124 1125]
#   [1131 1132 1133 1134 1135]
#   [1141 1142 1143 1144 1145]]]
# 
# 
# === preparing of W===
#(2, 1, 3, 3)
#[[[[3111 3112 3113]
#   [3121 3122 3123]
#   [3131 3132 3133]]]
#
#
# [[[4111 4112 4113]
#   [4121 4122 4123]
#   [4131 4132 4133]]]]

ここで、xの一部分((1,1)~(3,3)の部分)についてWで畳み込みをしようとすると、畳み込み演算結果は以下になる。
$$
    \begin{align}
    y = 1111 * 3111 + 1112 * 3112 + 1113 * 3113 + \\
        1121 * 3121 + 1122 * 3122 + 1123 * 3123 + \\
        1131 * 3131 + 1132 * 3132 + 1133 * 3133   
    \end{align}
$$
なるほど、この結果と★1(col)と★2(col_W)の行列積の結果を見比べると、確かに畳み込み演算が行列積で実現できていることがわかる。
ただ、一枚目の画像と２枚目のフィルタについても行列積をとっているのが気になる。これってテキストではこう説明されていたっけ？

### 出力画像(データ）の取得処理。最後の軸の入れ替え

In [17]:
out = np.dot(col, col_W) # この辺の処理の理解のためには、バイアスはあえて省いても良いかと思われ
print(">>> col * col_W")
print(out.shape)
print(out)
out = out.reshape(N, out_h, out_w, -1)
print(">>> out.reshape(N, out_h=%d, out_w=%d,-1)"%(out_h, out_w))
print(out.shape)
print(out)
out = out.transpose(0, 3, 1, 2)
print(">>> out.transpose(0,3,1,2)")
print(out.shape)
print(out)
#このtransposeはもともとの軸(N,H,W,C)を(N,C,H,W)となるように変更する
#      もとものindex->       0 1 2 3 => 0 3 1 2

>>> col * col_W
(12, 2)
[[31526562. 41624562.]
 [31554660. 41661660.]
 [31582758. 41698758.]
 [31807542. 41995542.]
 [31835640. 42032640.]
 [31863738. 42069738.]
 [59624562. 78722562.]
 [59652660. 78759660.]
 [59680758. 78796758.]
 [59905542. 79093542.]
 [59933640. 79130640.]
 [59961738. 79167738.]]
>>> out.reshape(N, out_h=2, out_w=3,-1)
(2, 2, 3, 2)
[[[[31526562. 41624562.]
   [31554660. 41661660.]
   [31582758. 41698758.]]

  [[31807542. 41995542.]
   [31835640. 42032640.]
   [31863738. 42069738.]]]


 [[[59624562. 78722562.]
   [59652660. 78759660.]
   [59680758. 78796758.]]

  [[59905542. 79093542.]
   [59933640. 79130640.]
   [59961738. 79167738.]]]]
>>> out.transpose(0,3,1,2)
(2, 2, 2, 3)
[[[[31526562. 31554660. 31582758.]
   [31807542. 31835640. 31863738.]]

  [[41624562. 41661660. 41698758.]
   [41995542. 42032640. 42069738.]]]


 [[[59624562. 59652660. 59680758.]
   [59905542. 59933640. 59961738.]]

  [[78722562. 78759660. 78796758.]
   [79093542. 79130640. 79167738.]]]]


## 終わりに
ここまでで、Convolutionクラスのforwardが何をやっているのかについて、概要レベルで学ぶことができた。
だが、transposeする前の演算でなぜ軸が(N,H,W,C)に変わったのかが良くわからない。また、backwardについても理解するべきだろうが、Convolutionレイヤの微分については見た感じ情報量が少ない感じで（まともに探していないだけ）、理解が出来ない。こういった状態であるが、あえて深追いはしないでおく。この辺は原論文を見ればわかるのだろうか。
画像認識関連、機械学習エンジニアのプロになるのであれば、より深い学習が必要だと思われるので、TODOリストに乗せて、必要性があれば追加学習していくとする。

# 参考URL
1. 基本的な理解(im2col/col2imの図解がわかりやすい)　https://qiita.com/t-tkd3a/items/6b17f296d61d14e12953
2. 行列積による畳込み(im2colした後の行列サイズやフィルターの解説についてわかりやすい) https://www.youtube.com/watch?v=PWPJVws7l0M